In [2]:
%cd /home/tirthankar-mittra/PhysicsWordProblem/NOVELTY_SCORE/
!pwd


\\wsl.localhost\Ubuntu-24.04\home\tirthankar-mittra\PhysicsWordProblem\NOVELTY_SCORE


'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import re 
from collections import defaultdict
import math 

def bleu_score(s_ref, s, upper_limit = 5, REF = 40):
  s_ref_list = [word.lower() for word in s_ref.split()]
  s_list = [word.lower() for word in s.split()]
  def ngram(sentence, gram):
    n = len(sentence)
    n_gram_dict = defaultdict(int)
    for idx in range(n - gram + 1):
       n_gram_dict[" ".join(sentence[idx:idx + gram])] += 1
    return n_gram_dict
  score = 0.0
  for gram in range(2, upper_limit + 1):
    n_gram_score = 0
    s_ref_ngram = ngram(s_ref_list, gram)
    s_ngram = ngram(s_list, gram)
    for phrase in s_ngram:
      if phrase in s_ref_ngram:
        n_gram_score += min(s_ref_ngram[phrase], s_ngram[phrase])
    score += math.log((1 + n_gram_score) / (1 + len(s_ngram)))
  c1, c2 = len(s_ref_list), len(s_list)
  cavg = (c1 + c2) // 2
  bp = min(1, math.exp(1 - REF/cavg))
  return round(bp * math.exp(score), 6)

In [4]:
def parse_data(filename):
    question_arr = []
    with open(f"{filename}", "r") as file:
        question = ""
        pattern = r"\b\d+(\.\d+)?([eE][+-]?\d+)?\b"
        stop_signs = ['.', ',', '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '<', '>', '\\', '|', '`', '~', '@', '#', '$', '%', '&', '*', '-', '_', '+', '=', '"', "'"]   
        for line in file:
            for ch in line:
                if ch == '{': 
                    question = "" 
                elif ch == '}':
                    question_arr.append(question)
                else:  question += ch
        for idx, question in enumerate(question_arr):
            question_arr_temp = []
            question = question.strip()
            for stop_sign in stop_signs:
                question = question.replace(stop_sign, "")
            question = re.sub(pattern, "X", question)
            for word in question.split():
                question_arr_temp.append(word.lower())
            question_arr[idx] = " ".join(question_arr_temp)
    return question_arr


In [5]:
model_names = ["own", "deepseek", "mistral", "llama", "gpt3.5turbo"]
first_col_size = 30
for model_name in model_names:
    qarr = parse_data(f"{model_name}.dat")
    total_bs, qn = 0, len(qarr)
    for i in range(qn):
        for j in range(i+1, qn):
            total_bs += bleu_score(qarr[i], qarr[j])
    prefix = f'Conformity Score[{model_name}]:'
    print(f'{prefix}' + ' ' * (first_col_size - len(prefix)) +\
          f'{round(total_bs, 4)}')

Conformity Score[own]:        0.0914
Conformity Score[deepseek]:   0.2442
Conformity Score[mistral]:    1.0336
Conformity Score[llama]:      0.4539
Conformity Score[gpt3.5turbo]:0.1206
